## Transaction Data Visualisation

In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL Project")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/17 16:19:49 WARN Utils: Your hostname, DESKTOP-HS8OQ4F resolves to a loopback address: 127.0.1.1; using 172.23.159.30 instead (on interface eth0)
22/09/17 16:19:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/17 16:19:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load Dataset

In [2]:
dataset = spark.read.parquet("../../data/curated/process_data.parquet/")

## Outliers

In [9]:
category = dataset.select('category').distinct()
subcategory = dataset.select('category').distinct()
for cat in category.collect():
    subset = dataset.filter(F.col("category") == cat[0])
    iqr = 
    min = 
    max = 
        

retail_and_wholesale_trade
rental_hiring_and_real_estate
arts_and_recreation
others
info_media_and_telecommunications


## Distributions of Variables with and without Outliers

## Relationship between Features